imports

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import random
import re
from tools import retriever_products, write_order_to_file, propose_alternatives, write_ocomplaint_to_file, search_orders, get_latest_orders_for_user
from prompts import system_message, user_prompt
import gradio as gr

OpenAi and model configuration

In [ ]:
# openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# MODEL = "llama3.2"

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins with {openai_api_key[:8]}")
else:
    print("OpenAI API Key? As if!")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()
HISTORY_FILE = "jsons/chat_history.json"

OpenAI API Key exists and begins with sk-proj-


In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retriever_products",
            "description": "Searches a pre-built FAISS vector database for pharmaceutical product information. "
            "Call this whenever a product is mentioned.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The name or type of pharmaceutical product to search for."
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "write_order_to_file",
            "description": "Logs a user's pharmaceutical product order to a file in JSON format. Call this whenever a user confirms an order.",
            "parameters": {
                "type": "object",
                "properties": {
                    "UserSubId": {
                        "type": "string",
                        "description": "The identifier or name of the user placing the order."
                    },
                    "ProductId": {
                        "type": "string",
                        "description": "The identifier of the product ordered."
                    },
                    "ProductName": {
                        "type": "string",
                        "description": "The name of the pharmaceutical product being ordered."
                    },
                    "Qty": {
                        "type": "number",
                        "description": "The number of units the user wants to order."
                    },
                    "TotalAmount": {
                        "type": "number",
                        "description": "The total cost of the order in the local currency."
                    }
                },
                "required": ["UserSubId", "ProductId", "ProductName", "Qty", "TotalAmount"],
                "additionalProperties": False
            }
        }
    },
    {
    "type": "function",
    "function": {
        "name": "write_ocomplaint_to_file",
        "description": "Logs a customer's complaint in JSON format. Use this when a user expresses dissatisfaction or files a complaint related to products or services.",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier of the user submitting the complaint."
                },
                "category": {
                    "type": "string",
                    "description": "The category of the complaint (Delivery claim, Commercial claim, Recovery claim)."
                },
                "subcategory": {
                    "type": "string",
                    "description": "The sub category of the complaint (Driver, Salesperson, Recovery team)."
                },
                "description": {
                    "type": "string",
                    "description": "The full text of the complaint expressed by the user."
                }
            },
            "required": ["customer_id", "category", "subcategory", "description"]
        }
    }
},
{
  "type": "function",
  "function": {
    "name": "search_orders",
    "description": "Searches the FAISS vector database for pharmaceutical order information. Use this to track or find details about orders based on product name, order ID, status (e.g., livrée), or order date.",
    "parameters": {
      "type": "object",
      "properties": {
        "query": {
          "type": "string",
          "description": "The search term related to an order, such as product name, order status (like 'confirmée', 'en livraison'), order ID, or order date."
        }
      },
      "required": ["query"]
    }
  }
},
{
        "type": "function",
        "function": {
            "name": "get_latest_orders_for_user",
            "description": "Returns the 5 most recent orders placed by a given user. Use this to retrieve a user's latest purchase activity.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "string",
                        "description": "The identifier of the user to look up recent orders for."
                    }
                },
                "required": ["user_id"]
            }
        }
    }

]


In [ ]:
import json
from datetime import datetime

def handle_tool_call(message):
    responses = []

    for tool_call in message.tool_calls:
        try:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            # Log the tool call
            log_entry = {
                "timestamp": datetime.utcnow().isoformat(),
                "tool": function_name,
                "arguments": arguments
            }
            with open("jsons/tool_logs.jsonl", "a") as f:
                f.write(json.dumps(log_entry) + "\n")

            # Process the tool call
            if function_name == 'retriever_products':
                query = arguments["query"]
                print(f"retriever_products called with arguments: {query}")
                outdata = retriever_products(query)
                responses.append({
                    "role": "tool",
                    "content": json.dumps({
                        "query": query,
                        "documents": outdata
                    }),
                    "tool_call_id": tool_call.id
                })

            elif function_name == 'write_order_to_file':
                print(f"write_order_to_file called with arguments: {arguments}")
                write_order_to_file(
                    user_ID=arguments["UserSubId"],
                    product_ID=arguments["ProductId"],
                    product_name=arguments["ProductName"],
                    quantity=arguments["Qty"],
                    total_price=arguments["TotalAmount"]
                )
                responses.append({
                    "role": "tool",
                    "content": json.dumps({
                        "status": "order recorded",
                        "details": arguments
                    }),
                    "tool_call_id": tool_call.id
                })

            elif function_name == 'propose_alternatives':
                dci_query = arguments["dci_query"]
                print(f"propose_alternatives called with DCI query: {dci_query}")
                alternatives = propose_alternatives(dci_query)
                responses.append({
                    "role": "tool",
                    "content": json.dumps({
                        "dci_query": dci_query,
                        "alternatives": alternatives
                    }),
                    "tool_call_id": tool_call.id
                })

            elif function_name == 'write_ocomplaint_to_file':
                print(f"write_complaint_to_file called with arguments: {arguments}")
                write_ocomplaint_to_file(
                    customer_id=arguments["customer_id"],
                    category=arguments["category"],
                    sub_category=arguments["subcategory"],
                    complaint_text=arguments["description"]
                )
                responses.append({
                    "role": "tool",
                    "content": json.dumps({
                        "status": "complaint recorded",
                        "details": arguments
                    }),
                    "tool_call_id": tool_call.id
                })

            elif function_name == 'search_orders':
                query = arguments["query"]
                print(f"search_orders called with query: {query}")
                results = search_orders(query)
                responses.append({
                    "role": "tool",
                    "content": json.dumps({
                        "query": query,
                        "results": json.loads(results)
                    }),
                    "tool_call_id": tool_call.id
                })

            elif function_name == 'get_latest_orders_for_user':
                user_id = arguments["user_id"]
                print(f"get_latest_orders_for_user called for user_id: {user_id}")
                latest_orders = get_latest_orders_for_user(user_id)
                responses.append({
                    "role": "tool",
                    "content": json.dumps({
                        "user_id": user_id,
                        "latest_orders": latest_orders
                    }),
                    "tool_call_id": tool_call.id
                })

            else:
                # Handle unknown tools
                print(f"[ERROR] Unknown tool: {function_name}")
                responses.append({
                    "role": "tool",
                    "content": json.dumps({
                        "status": "error",
                        "message": f"Unknown tool: {function_name}"
                    }),
                    "tool_call_id": tool_call.id
                })

        except Exception as e:
            print(f"[ERROR] Exception during tool processing: {e}")
            responses.append({
                "role": "tool",
                "content": json.dumps({
                    "status": "error",
                    "message": f"Exception in tool '{tool_call.function.name}': {str(e)}"
                }),
                "tool_call_id": tool_call.id
            })

    return responses


In [5]:
# --- History handlers ---
def save_history(history):
    with open(f"{HISTORY_FILE}", "w") as f:
        json.dump(history, f)

def load_history():
    try:
        with open(f"{HISTORY_FILE}", "r") as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return []


In [ ]:
import gradio as gr
import pandas as pd
import plotly.express as px
from datetime import datetime


history = [
    {
        "role": "system",
        "content": (
            "If the product is not found so not show a list of related options just say it is not available"
            "The ProductId should always be included in the product information and used when writing an order. "
            "Only ask the user to provide you with their id if it is never been mentioned in the session, else use the one mentioned by the user and only the one mentioned by the user"
            "An order should only be written once; if followed by another order, only the new one should be recorded."
            "A claim should only be written once; if followed by another claim, only the new one should be recorded."
            "If the date provided when tracking an order is written as (26 april 2025) turn it into this format: dd-mm-yyyy before passing it to the tool."
            "try to get as much informations as possible before registering a claim and use them to write a more detailed description"
            "never write a claim twice if followed by another claim"
        )
    }
]

def chat(history):
    # Ensure tools is available in the function
    global tools
    
    messages = [{"role": "system", "content": system_message}] + history
    if tools is None:
        tools = []  # Default to empty list if tools is not provided
    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # Vérifie si le modèle a demandé l’appel d’un outil 
    if response.choices[0].finish_reason == "tool_calls":
        # Si un ou plusieurs outils sont demandés :
        # on récupère la requête,
        # on appelle les outils via la fonction handle_tool_call,
        # on ajoute les messages (l’appel + les résultats) à l’historique complet.
        message = response.choices[0].message
        tool_responses = handle_tool_call(message)  #get the full list
        messages.append(message)
        messages.extend(tool_responses)  #extend the list with full dicts

        # Re-lance un nouvel appel à l’API avec les nouvelles informations/outils, pour que l’assistant réponde à l’utilisateur en tenant compte des résultats.
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    reply = response.choices[0].message.content

    history.append({"role": "assistant", "content": reply})
    save_history(history)

    
    return history
import gradio as gr
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import json

# === Load and preprocess data ===
def load_data():
    global logs, orders, claims
    # Load tool logs
    logs = pd.read_json("jsons/tool_logs.jsonl", lines=True)
    logs["timestamp"] = pd.to_datetime(logs["timestamp"])

    # Load orders and claims data
    orders = pd.read_json('jsons/orders.json', lines=True)
    claims = pd.read_json('jsons/claims.json', lines=True)

    # Process datetime columns
    orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
    claims['created_at'] = pd.to_datetime(claims['created_at'])

    # Create time-based aggregations
    claims['YearMonth'] = claims['created_at'].dt.to_period('M').astype(str)
    orders['Month'] = orders['OrderDate'].dt.month
    orders['MonthName'] = orders['OrderDate'].dt.strftime('%B')

# Initial data load
load_data()

# === Visualization Functions ===
def get_empty_fig(title=""):
    fig = go.Figure()
    fig.update_layout(title=title)
    return fig

def create_metric_card(title, value, unit=""):
    return gr.Markdown(
        f"""
        <div style="
            border: 1px solid #e0e0e0;
            border-radius: 8px;
            padding: 15px;
            margin: 10px;
            text-align: center;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            background: white;
        ">
            <h3 style="margin: 0; color: #555;">{title}</h3>
            <p style="font-size: 24px; margin: 10px 0; font-weight: bold; color: #333;">
                {value}{unit}
            </p>
        </div>
        """,
        visible=True
    )

def get_fig_most_ordered():
    most_ordered = orders.groupby("ProductName")["Qty"].sum().sort_values(ascending=False).reset_index()
    
    fig = px.bar(
        most_ordered,
        x="Qty",
        y="ProductName",
        orientation='h',
        title="Most Ordered Products by Quantity",
        color="Qty",
        color_continuous_scale=px.colors.sequential.Teal,
        labels={"Qty": "Quantity"},  # Only label the x-axis
    )
    
    # Customize hover and layout
    fig.update_traces(
        hovertemplate="<b>%{y}</b><br>Quantity: %{x:.0f}<extra></extra>",
        marker_line_color="#A3D9C1", 
        marker_line_width=1.5
    )
    
    fig.update_layout(
        margin=dict(l=180, r=40, t=60, b=40),
        xaxis_title="Quantity Ordered",
        yaxis_title=None,  # Explicitly remove y-axis label
        yaxis=dict(showticklabels=True),  # Keep product names visible
        hoverlabel=dict(
            bgcolor="white",
            font_size=12
        )
    )
    
    return fig

def get_fig_monthly_total():
    monthly_sum = orders.groupby(["Month", "MonthName"])["TotalAmount"].sum().reset_index().sort_values("Month")
    fig = px.bar(
        monthly_sum,
        x="MonthName",
        y="TotalAmount",
        title="Total Amount Ordered Per Month",
        labels={"TotalAmount": "Total Amount (DA)"},
        color="TotalAmount",
        color_continuous_scale=px.colors.sequential.Magma,
    )
    return fig

def get_fig_monthly_orders():
    monthly_orders = orders.groupby(["Month", "MonthName"]).size().reset_index(name="NumberOfOrders").sort_values("Month")
    fig = px.bar(
        monthly_orders,
        x="MonthName",
        y="NumberOfOrders",
        title="Total Number of Orders Per Month",
        labels={"NumberOfOrders": "Number of Orders"},
        color="NumberOfOrders",
        color_continuous_scale=px.colors.sequential.Viridis,
    )
    return fig

def get_fig_top_products():
    top_products_per_month = (
        orders.groupby(['Month', 'MonthName', 'ProductName'])
        .size()
        .reset_index(name='OrderCount')
        .sort_values(['Month', 'OrderCount'], ascending=[True, False])
        .drop_duplicates(subset=['Month'])
    )
    fig = px.bar(
        top_products_per_month,
        x="MonthName",
        y="OrderCount",
        color="ProductName",
        title="Top Ordered Product Per Month",
        text="ProductName",
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    fig.update_traces(textposition="outside")
    return fig

def get_fig_claim_category():
    category_counts = claims['category'].value_counts().reset_index()
    category_counts.columns = ['Category', 'Count']
    fig = px.bar(
        category_counts,
        x='Category',
        y='Count',
        title='Most Claimed Categories',
        labels={'Count': 'Number of Claims'},
        color='Count',
        color_continuous_scale=px.colors.sequential.Sunset
    )
    fig.update_traces(marker_line_color='#F0E5D8', marker_line_width=1.5)
    return fig

def get_fig_claim_subcategory():
    subcategory_counts = claims['subcategory'].value_counts().reset_index()
    subcategory_counts.columns = ['Subcategory', 'Count']
    fig = px.bar(
        subcategory_counts,
        x='Subcategory',
        y='Count',
        title='Most Claimed Subcategories',
        labels={'Count': 'Number of Claims'},
        color='Count',
        color_continuous_scale=px.colors.sequential.Agsunset
    )
    fig.update_traces(marker_line_color='#E3E3E3', marker_line_width=1.5)
    fig.update_layout(xaxis_tickangle=-45)
    return fig

def get_fig_claims_monthly():
    monthly_claims = claims.groupby('YearMonth').size().reset_index(name='ClaimsCount')
    fig = px.bar(
        monthly_claims,
        x='YearMonth',
        y='ClaimsCount',
        title='Total Number of Claims Per Month',
        labels={'ClaimsCount': 'Number of Claims'},
        color='ClaimsCount',
        color_continuous_scale=px.colors.sequential.Peach
    )
    fig.update_traces(marker_line_color='#FBE3D2', marker_line_width=1.5)
    return fig

def get_fig_tool_usage():
    tool_counts = logs["tool"].value_counts().reset_index()
    tool_counts.columns = ["Tool", "UsageCount"]
    fig = px.bar(
        tool_counts,
        x="Tool",
        y="UsageCount",
        title="Most Used Tools",
        labels={"UsageCount": "Number of Uses", "Tool": "Tool Name"},
        color="UsageCount",
        color_continuous_scale=px.colors.sequential.Blues
    )
    fig.update_traces(marker_line_color='black', marker_line_width=1)
    return fig

def get_fig_dci():
    suggester_calls = logs[logs["tool"] == "propose_alternatives"].copy()
    suggester_calls["dci"] = suggester_calls["arguments"].apply(lambda x: x.get("query", "").strip().upper())
    dci_counts = suggester_calls["dci"].value_counts().reset_index()
    dci_counts.columns = ["DCI", "Count"]
    fig = px.bar(
        dci_counts.head(20),
        x="DCI",
        y="Count",
        title="Most Proposed Alternatives (DCI) via propose_alternatives",
        labels={"Count": "Number of Proposals", "DCI": "Active Substance (DCI)"},
        color="Count",
        color_continuous_scale=px.colors.sequential.Viridis
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.update_traces(marker_line_color='black', marker_line_width=1)
    return fig

def get_fig_hourly():
    logs["hour"] = logs["timestamp"].dt.hour
    hour_counts = logs["hour"].value_counts().sort_index().reset_index()
    hour_counts.columns = ["HourOfDay", "Count"]
    fig = px.bar(
        hour_counts,
        x="HourOfDay",
        y="Count",
        title="Agent Usage Distribution by Hour of Day",
        labels={"HourOfDay": "Hour (24h)", "Count": "Number of Calls"},
        color="Count",
        color_continuous_scale=px.colors.sequential.Plasma
    )
    fig.update_traces(marker_line_color='black', marker_line_width=1)
    return fig

def get_fig_most_searched():
    retriever_calls = logs[logs["tool"] == "retriever_products"].copy()
    retriever_calls["product_query"] = retriever_calls["arguments"].apply(lambda x: x.get("query", "").strip())
    product_counts = retriever_calls["product_query"].value_counts().reset_index()
    product_counts.columns = ["ProductQuery", "SearchCount"]
    fig = px.bar(
        product_counts.head(20),
        x="ProductQuery",
        y="SearchCount",
        title="Top 20 Most Searched Products by retriever_products",
        labels={"SearchCount": "Number of Searches", "ProductQuery": "Product Name / Query"},
        color="SearchCount",
        color_continuous_scale=px.colors.sequential.Magenta
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.update_traces(marker_line_color='black', marker_line_width=1)
    return fig

def get_fig_search_vs_order():
    retriever_calls = logs[logs["tool"] == "retriever_products"].copy()
    retriever_calls["Product"] = retriever_calls["arguments"].apply(lambda x: x.get("query", "").strip().lower())
    search_counts = retriever_calls["Product"].value_counts().reset_index()
    search_counts.columns = ["Product", "SearchCount"]

    order_calls = logs[logs["tool"] == "write_order_to_file"].copy()
    order_calls["Product"] = order_calls["arguments"].apply(lambda x: x.get("ProductName", "").strip().lower())
    order_counts = order_calls["Product"].value_counts().reset_index()
    order_counts.columns = ["Product", "OrderCount"]

    merged = pd.merge(search_counts, order_counts, on="Product", how="outer").fillna(0)
    melted = merged.melt(id_vars="Product", value_vars=["SearchCount", "OrderCount"],
                         var_name="Action", value_name="Count")
    melted["Total"] = melted.groupby("Product")["Count"].transform("sum")
    melted = melted.sort_values(by="Total", ascending=False).drop(columns="Total")
    top_n = 20
    top_products = melted["Product"].unique()[:top_n]
    melted = melted[melted["Product"].isin(top_products)]

    fig = px.bar(
        melted,
        x="Product",
        y="Count",
        color="Action",
        barmode="group",
        title="Search vs Order Count per Product",
        labels={"Count": "Number of Calls", "Product": "Product Name", "Action": "Action Type"},
        color_discrete_map={"SearchCount": "blue", "OrderCount": "green"}
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.update_traces(marker_line_color='black', marker_line_width=1)
    return fig

def get_fig_daily():
    logs["date"] = logs["timestamp"].dt.date.astype(str)
    day_counts = logs["date"].value_counts().sort_index().reset_index()
    day_counts.columns = ["Date", "Count"]
    fig = px.bar(
        day_counts,
        x="Date",
        y="Count",
        title="Agent Usage by Day",
        labels={"Date": "Date", "Count": "Number of Calls"},
        color="Count",
        color_continuous_scale=px.colors.sequential.Teal
    )
    fig.update_traces(marker_line_color='black', marker_line_width=1)
    return fig

def get_fig_claims_today():
    today = pd.Timestamp.now().normalize()
    count_today = claims[claims['created_at'] >= today].shape[0]
    fig = px.bar(
        x=["Today"],
        y=[count_today],
        title="Total Claims Today",
        text=[count_today],
        labels={"x": "Date", "y": "Number of Claims"},
    )
    fig.update_traces(
        marker_color="#FEC8D8",
        marker_line_color="#FEEAE6",
        marker_line_width=1.5,
        textposition="outside"
    )
    fig.update_layout(
        height=300,
        showlegend=False,
        margin=dict(l=20, r=20, t=50, b=20)
    )
    return fig

df = pd.read_csv("data/orderlines_with_status.csv", parse_dates=["OrderDate"])

def get_fig_order_status_distribution(df):
    # Use first status per order to avoid duplicates
    status_counts = df.groupby("RefOrderId").first()["status"].value_counts().reset_index()
    status_counts.columns = ["Status", "Count"]

    fig = px.bar(
        status_counts,
        x="Status",
        y="Count",
        title="Order Status Distribution",
        text="Count",
        labels={"Count": "Number of Orders"}
    )

    fig.update_traces(
        marker_color="#B5EAD7",
        marker_line_color="white",
        marker_line_width=1.5,
        textposition="outside"
    )
    fig.update_layout(
        height=350,
        showlegend=False,
        margin=dict(l=20, r=20, t=50, b=20)
    )

    return fig

def get_fig_top_5_customers_by_total(filepath="orders.json"):
    # Load JSONL file
    with open(filepath, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]

    df = pd.DataFrame(data)

    # Group by customer and sum their total purchases
    customer_totals = (
        df.groupby("UserSubId")["TotalAmount"]
        .sum()
        .reset_index()
        .nlargest(5, "TotalAmount")  # Keep top 5
    )

    # Plot
    fig = px.bar(
        customer_totals,
        x="UserSubId",
        y="TotalAmount",
        title="Top 5 Customers by Total Order Value",
        text="TotalAmount",
        labels={"UserSubId": "Customer ID", "TotalAmount": "Total Amount (DA)"}
    )

    fig.update_traces(
        marker_color="#CBAACB",
        marker_line_color="white",
        marker_line_width=1.5,
        textposition="outside"
    )

    fig.update_layout(
        height=350,
        showlegend=False,
        margin=dict(l=20, r=20, t=50, b=20)
    )

    return fig
####
def get_fig_claims_by_customer(df):
    claim_counts = df['customer_id'].value_counts().reset_index()
    claim_counts.columns = ['Customer ID', 'Claims']

    fig = px.bar(
        claim_counts.head(5),  # Top 5 customers
        x='Customer ID',
        y='Claims',
        title='Top 5 Customers by Number of Claims',
        text='Claims'
    )
    fig.update_traces(marker_color='#B5EAD7', textposition='outside')
    fig.update_layout(showlegend=False, height=350, margin=dict(l=20, r=20, t=50, b=20))
    return fig

def get_fig_claims_daily_trend(df):
    daily = df.groupby(df['created_at'].dt.date).size().reset_index(name='Claims')
    fig = px.line(
        daily,
        x='created_at',
        y='Claims',
        markers=True,
        title='Daily Claims Volume'
    )
    fig.update_traces(line_color='#FFB7B2')
    fig.update_layout(height=350, margin=dict(l=20, r=20, t=50, b=20))
    return fig

def get_fig_claim_category_pie(df):
    category_counts = df['category'].value_counts().reset_index()
    category_counts.columns = ['Category', 'Count']

    fig = px.pie(
        category_counts,
        names='Category',
        values='Count',
        title='Claim Distribution by Category',
        hole=0.4  # Donut style
    )

    fig.update_traces(textinfo='percent+label', marker=dict(line=dict(color='#fff', width=2)))
    fig.update_layout(margin=dict(t=50, b=20, l=20, r=20))
    return fig



# --- Analytics View Routing ---
def analytics_view(selection):
    if selection == "Orders":
        return [
            get_fig_most_ordered(),
            get_fig_monthly_total(),
            get_fig_monthly_orders(),
            get_fig_top_products(),
            get_fig_order_status_distribution(df),
            get_fig_top_5_customers_by_total(filepath="orders.json"),
        ]
    elif selection == "Claims":
        return [
            get_fig_claims_monthly(),
            get_fig_claim_subcategory(),
            get_fig_claim_category(),
            get_fig_claims_by_customer(claims),
            get_fig_claims_daily_trend(claims),
            get_fig_claim_category_pie(claims)
        ]
    elif selection == "Tools":
        return [
            get_fig_tool_usage(),
            get_fig_dci(),
            get_fig_hourly(),
            get_fig_most_searched(),
            get_fig_search_vs_order(),
            get_fig_daily()
        ]
    return [get_empty_fig("Select an analytics category")] * 6

def update_dashboard_metrics(category):
    # Reload all data to ensure fresh counts
    load_data()
    
    # générer des indicateurs clés de la journée en cours
    today = pd.Timestamp.now().normalize()
    qty_today = orders[orders['OrderDate'] >= today].shape[0]
    amount_today = orders.loc[orders['OrderDate'] >= today, 'TotalAmount'].sum()
    claims_count = claims[claims['created_at'] >= today].shape[0]

    if category == "Orders":
        figs = [
            get_fig_most_ordered(),
            get_fig_monthly_total(),
            get_fig_monthly_orders(),
            get_fig_top_products(),
            get_fig_order_status_distribution(df),
            get_fig_top_5_customers_by_total(filepath="orders.json"),
        ]
    elif category == "Claims":
        figs = [
            get_fig_claims_monthly(),
            get_fig_claim_subcategory(),
            get_fig_claim_category(),
            get_fig_claims_by_customer(claims),
            get_fig_claims_daily_trend(claims),
            get_fig_claim_category_pie(claims)
        ]
    elif category == "Tools":
        figs = [
            get_fig_tool_usage(),
            get_fig_dci(),
            get_fig_hourly(),
            get_fig_most_searched(),
            get_fig_search_vs_order(),
            get_fig_daily()
        ]
    else:
        figs = [get_empty_fig("Select an analytics category")] * 6

    return [
        create_metric_card("Orders Today", qty_today),
        create_metric_card("Amount Today", f"{amount_today:,.2f}", " DA"),
        create_metric_card("Claims Today", claims_count),
        *figs
    ]

# === Gradio UI ===
with gr.Blocks(title="Pharmacy Analytics Dashboard", theme=gr.themes.Soft(), css="""
    .refresh-container {
        border: 1px solid #e0e0e0 !important;
        border-radius: 4px !important;
        padding: 10px !important;
        margin-bottom: 10px !important;
        background: white !important;
    }
    .refresh-button {
        background-color: #4285F4 !important;
        color: white !important;
        border: none !important;
        height: 36px !important;
        padding: 0 16px !important;
        border-radius: 4px !important;
        font-weight: 500 !important;
    }
    .refresh-button:hover {
        background-color: #3367D6 !important;
    }
    """) as demo:
    with gr.Tabs():
        with gr.Tab("📊 Analytics Dashboard"):
            # Container for dropdown and refresh button
            with gr.Column(elem_classes="refresh-container"):
                with gr.Row():
                    category = gr.Dropdown(
                        choices=["Orders", "Claims", "Tools"],
                        label="Select Category",
                        value="Orders"
                    )
                with gr.Row():
                    gr.HTML("""<div style="flex-grow: 1;"></div>""")  # Spacer
                    refresh_btn = gr.Button(
                        "Refresh Dashboards",
                        elem_classes="refresh-button"
                    )
            
            # Metrics cards row
            with gr.Row():
                card1 = gr.Markdown()
                card2 = gr.Markdown()
                card3 = gr.Markdown()

            # Plots row
            with gr.Row():
                with gr.Column():
                    plot1 = gr.Plot()
                    plot2 = gr.Plot()
                with gr.Column():
                    plot3 = gr.Plot()
                    plot4 = gr.Plot()
                with gr.Column():
                    plot5 = gr.Plot()
                    plot6 = gr.Plot()

            # Initial load
            demo.load(
                lambda: update_dashboard_metrics(category.value),
                inputs=None,
                outputs=[card1, card2, card3, plot1, plot2, plot3, plot4, plot5, plot6]
            )

            # Category change
            category.change(
                analytics_view,
                inputs=category,
                outputs=[plot1, plot2, plot3, plot4, plot5, plot6]
            )

            # Refresh button action
            refresh_btn.click(
                fn=update_dashboard_metrics,
                inputs=category,
                outputs=[card1, card2, card3, plot1, plot2, plot3, plot4, plot5, plot6]
            )

        with gr.Tab("💬 Assistant Chat"):
            with gr.Row():
                chatbot = gr.Chatbot(height=500, type="messages")
            with gr.Row():
                entry = gr.Textbox(label="Chat with our AI Assistant:")

            def do_entry(message, history):
                history += [{"role":"user", "content":message}]
                return "", history
            entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
                chat, inputs=chatbot, outputs=[chatbot]
            )
if __name__ == "__main__":
    demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


retriever_products called with arguments: doliprane


c:\Users\THINKPAD T470\Desktop\pfe\tools.py:8: LangChainDeprecationWarning:

The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.

